In [3]:
import pandas as pd
from io import StringIO
import requests
from datetime import datetime, timedelta

In [4]:
num_days_back = 30
# start_date = '20240923'

In [40]:
def fetch_data_for_dates(num_days_back, start_date=None):
    """
    Fetch data for the latest date back to a number of days (num_days_back)
    """

    if start_date is None:
        start_date = datetime.today().strftime('%Y%m%d')

    all_dataframes = []
    current_date = datetime.strptime(str(start_date), '%Y%m%d')

    for i in range(num_days_back + 1):
        as_of_date = current_date.strftime('%Y%m%d')
        url = f'https://www.ishares.com/us/products/239467/ishares-tips-bond-etf/1467271812596.ajax?fileType=csv&fileName=TIP_holdings&dataType=fund&asOfDate={as_of_date}'
        response = requests.get(url)
        
        if response.status_code == 200:
            try:
                csv_data = StringIO(response.text)
                df = pd.read_csv(csv_data, skiprows=9)  # Need to skip because header formats are messed up

                # If the dataframe is empty, raise an error
                if df.empty:
                    raise pd.errors.EmptyDataError
                
                df = df[df['Name'] == 'TREASURY (CPI) NOTE']
                df['As_of'] = as_of_date
                
                all_dataframes.append(df)
            except pd.errors.EmptyDataError:
                print(f"No data available for {as_of_date}.")
        else:
            print(f"Failed to fetch data for {as_of_date}. Status code: {response.status_code}")
        
        current_date -= timedelta(days=1)
    
    # Concatenate all dataframes into one
    if all_dataframes:
        final_df = pd.concat(all_dataframes, ignore_index=True)
        final_df['Market Value'] = final_df['Market Value'].replace({',': ''}, regex=True).astype('float')/1000000
        final_df['Market Value (in MM)'] = final_df['Market Value'].round(1)
        final_df['Maturity'] = pd.to_datetime(final_df['Maturity'])
        final_df['Coupon (%)'] = final_df['Coupon (%)'].astype(str) + '%' 

        final_df = final_df.pivot_table(
            columns='As_of',
            values='Market Value (in MM)', 
            index=['Maturity', 'Coupon (%)']
        )

        final_df = final_df.sort_index(level='Maturity')
        final_df.index = final_df.index.set_levels(final_df.index.levels[0].strftime('%m/%d/%Y'), level=0)
        final_df.columns = pd.to_datetime(final_df.columns).strftime('%m/%d/%Y')
        return final_df
    else:
        return pd.DataFrame()  


In [44]:
final_data = fetch_data_for_dates(num_days_back=num_days_back)

No data available for 20240923.
No data available for 20240922.
No data available for 20240921.
No data available for 20240915.
No data available for 20240914.
No data available for 20240908.
No data available for 20240907.
No data available for 20240902.
No data available for 20240901.
No data available for 20240831.
No data available for 20240825.
No data available for 20240824.


In [45]:
final_data = final_data[sorted(final_data.columns, reverse=True)]
latest_col = final_data.columns[0]  
oldest_col = final_data.columns[-1]  

# Calculate the difference between the latest and oldest values
final_data[f'{num_days_back}_chg'] = final_data[latest_col] - final_data[oldest_col]
cols = final_data.columns.tolist()  
cols.insert(1, cols.pop(cols.index(f'{num_days_back}_chg'))) 
final_data = final_data[cols].iloc[:, :6]
timestamp = datetime.now().strftime('%Y%m%d')
final_data.to_excel(f'ishare_holdings_{timestamp}.xlsx')